In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal import graph
from sklearn import ensemble, metrics, model_selection, preprocessing

We'll be using data reflecting the morphology of Prague.

In [ ]:
building_data = gpd.read_file("data/prg_building_locations.gpkg", engine="pyogrio")

In [ ]:
building_data.columns

Let's use 20k to train the model on.

Some columns are independent variables.

Most of the models need the data standardised.

In [ ]:
independent_variables = ['floor_area_ratio', 'height', 'compactness',
       'street_alignment', 'interbuilding_distance',
       'block_perimeter_wall_length']

In [ ]:
building_data[independent_variables] = preprocessing.robust_scale(building_data[independent_variables])

In [ ]:
training_sample = building_data.sample(20_000, random_state=0)

In [ ]:
independent = training_sample[independent_variables]

## Train-test splits

Data need to be split into train and test subsets.

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(independent, training_sample["cluster"], test_size=.25, random_state=0)

We can start with the default model.

In [ ]:
model = ensemble.RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)


Get prediction. With classificaiotn, you have two options. The class:

In [ ]:
pred = model.predict(X_test)

Or probabilities. Class is just using max prob.

In [ ]:
proba = model.predict_proba(X_test)

## Evaluation

We may want to know accuracy

In [ ]:
accuracy = metrics.accuracy_score(pred, y_test)
accuracy

Or Kappa

In [ ]:
kappa = metrics.cohen_kappa_score(pred, y_test)
kappa

### Cross-validated prediction

To see a map without leakage.

In [ ]:
predicted  = model_selection.cross_val_predict(model, independent, training_sample["cluster"], cv=4)

Plot on a map.

In [ ]:
training_sample.explore(predicted, categorical=True, tiles="Cartodb positron", prefer_canvas=True)

Plot a spatial pattern of errors

In [ ]:
training_sample.plot(predicted == training_sample["cluster"], categorical=True, figsize=(16, 16), markersize=.5, cmap="bwr_r", legend=True)

### Confusion matrix

In [ ]:
cm = metrics.confusion_matrix(y_test, pred)
cm

In [ ]:
metrics.ConfusionMatrixDisplay(cm).plot()

## Spatial cross-validation

In [ ]:
gkf = model_selection.StratifiedGroupKFold(n_splits=5)
splits = gkf.split(training_sample, training_sample.cluster, groups=pd.factorize(training_sample.NAZ_ZSJ)[0])

In [ ]:
split_label = np.empty(len(training_sample), dtype=float)
for i, (train, test) in enumerate(splits):
    split_label[test] = i
training_sample["split"] = split_label

In [ ]:
training_sample

In [ ]:
ax = training_sample.plot("split", categorical=True, figsize=(15, 15), markersize=.5)
training_sample.dissolve("NAZ_ZSJ").convex_hull.boundary.plot(ax=ax, color="k", linewidth=.5, markersize=0)
ax

Prepare new training data

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent.loc[train]
X_test = independent.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_spatial_cv = ensemble.RandomForestClassifier(random_state=0)
rf_spatial_cv.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_spatial_cv.predict(X_test)

In [ ]:
accuracy_spatial_cv = metrics.accuracy_score(pred, y_test)
accuracy_spatial_cv

In [ ]:
kappa_spatial_cv = metrics.cohen_kappa_score(pred, y_test)
kappa_spatial_cv

Compare to original:

In [ ]:
accuracy, kappa

Plot a spatial pattern of errors

In [ ]:
predicted  = model_selection.cross_val_predict(rf_spatial_cv, independent, training_sample["cluster"], cv=4)
training_sample.plot(predicted == training_sample["cluster"], categorical=True, figsize=(16, 16), markersize=.5, cmap="bwr_r", legend=True)

The result is worse on paper but it is now more spatially robust - it will generalise better on unseen data.

## Feature engineering

### Proximity vartiables

In [ ]:
old_town_square = gpd.tools.geocode("Old Town Square, Prague").to_crs(building_data.crs)

In [ ]:
distance = training_sample.distance(old_town_square.geometry.item())
training_sample["distance_to_old_town"] = preprocessing.robust_scale(distance)

In [ ]:
independent_distance = training_sample[independent_variables + ["distance_to_old_town"]]

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent_distance.loc[train]
X_test = independent_distance.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_distance = ensemble.RandomForestClassifier(random_state=0)
rf_distance.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_distance.predict(X_test)

In [ ]:
accuracy_distance = metrics.accuracy_score(pred, y_test)
accuracy_distance

In [ ]:
kappa_distance = metrics.cohen_kappa_score(pred, y_test)
kappa_distance

In [ ]:
accuracy_spatial_cv, kappa_spatial_cv

Number of points within a threshold

In [ ]:
distance_200 = graph.Graph.build_distance_band(training_sample, 200)
points_in = distance_200.cardinalities
training_sample["points_in_200m"] = preprocessing.robust_scale(points_in)

In [ ]:
independent_proximity = training_sample[independent_variables + ["distance_to_old_town", "points_in_200m"]]

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent_proximity.loc[train]
X_test = independent_proximity.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_proximity = ensemble.RandomForestClassifier(random_state=0)
rf_proximity.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_proximity.predict(X_test)

In [ ]:
accuracy_proximity = metrics.accuracy_score(pred, y_test)
accuracy_proximity

In [ ]:
kappa_proximity = metrics.cohen_kappa_score(pred, y_test)
kappa_proximity

In [ ]:
accuracy_spatial_cv, kappa_spatial_cv

## Information join

In [ ]:
price = gpd.read_file("https://martinfleischmann.net/sds/chapter_03/data/SED_CenovaMapa_p_shp.zip", engine="pyogrio")
price["CENA"] = price["CENA"].replace("N", None).astype('float')

In [ ]:
price.head()

In [ ]:
price.crs.equals(training_sample.crs)

In [ ]:
price = price.to_crs(training_sample.crs)

In [ ]:
training_sample_price = training_sample.sjoin(price[["CENA", "geometry"]].dropna(), how="left").dropna(subset=["CENA"])

In [ ]:
training_sample_price["price"] = preprocessing.robust_scale(training_sample_price["CENA"])

Another model with price included.

In [ ]:
independent_proximity_price = training_sample_price[independent_variables + ["distance_to_old_town", "points_in_200m", "price"]]

In [ ]:
train = training_sample_price["split"] != 0
test = training_sample_price["split"] == 0
X_train = independent_proximity_price.loc[train]
X_test = independent_proximity_price.loc[test]
y_train = training_sample_price["cluster"].loc[train]
y_test = training_sample_price["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_proximity_price = ensemble.RandomForestClassifier(random_state=0)
rf_proximity_price.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_proximity_price.predict(X_test)

In [ ]:
accuracy_proximity_price = metrics.accuracy_score(pred, y_test)
accuracy_proximity_price

In [ ]:
kappa_proximity_price = metrics.cohen_kappa_score(pred, y_test)
kappa_proximity_price

## Spatial dependence

### Include spatially lagged variables in the model

In [ ]:
distance_200_row = distance_200.transform("r")
lagged_variables = []
for var in independent_variables:
    training_sample[f"{var}_lag"] = distance_200_row.lag(training_sample[var])
    lagged_variables.append(f"{var}_lag")

In [ ]:
lagged_variables

In [ ]:
independent_lag = training_sample[independent_variables + lagged_variables]

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent_lag.loc[train]
X_test = independent_lag.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_lag_200m = ensemble.RandomForestClassifier(random_state=0)
rf_lag_200m.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_lag_200m.predict(X_test)

In [ ]:
accuracy_lag = metrics.accuracy_score(pred, y_test)
accuracy_lag

In [ ]:
kappa_lag = metrics.cohen_kappa_score(pred, y_test)
kappa_lag

## Spatial heterogneity

### Include x, y coordinates

In [ ]:
training_sample[["x", "y"]] = preprocessing.robust_scale(training_sample.get_coordinates())

In [ ]:
independent_coordinates = training_sample[independent_variables + ["x", "y"]]

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent_coordinates.loc[train]
X_test = independent_coordinates.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_coordinates = ensemble.RandomForestClassifier(random_state=0)
rf_coordinates.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_coordinates.predict(X_test)

In [ ]:
accuracy_coordinates = metrics.accuracy_score(pred, y_test)
accuracy_coordinates

In [ ]:
kappa_coordinates = metrics.cohen_kappa_score(pred, y_test)
kappa_coordinates

In [ ]:
accuracy, kappa

### Fixed effects

In [ ]:
training_sample.NAZ_KU.nunique()

In [ ]:
ax = training_sample.plot("cluster", categorical=True, figsize=(15, 15), markersize=.5)
training_sample.dissolve("NAZ_KU").convex_hull.boundary.plot(ax=ax, color="k", linewidth=.5, markersize=0)
ax

In [ ]:
dummies = pd.get_dummies(training_sample.NAZ_KU)
dummies.head()

In [ ]:
training_sample = pd.concat([training_sample, dummies], axis=1)

In [ ]:
training_sample[dummies.columns] = preprocessing.robust_scale(training_sample[dummies.columns])

In [ ]:
independent_fixed = training_sample[independent_variables + dummies.columns.tolist()]

In [ ]:
train = training_sample["split"] != 0
test = training_sample["split"] == 0
X_train = independent_fixed.loc[train]
X_test = independent_fixed.loc[test]
y_train = training_sample["cluster"].loc[train]
y_test = training_sample["cluster"].loc[test]

Fit a new model, with more robust split.

In [ ]:
rf_fixed = ensemble.RandomForestClassifier(random_state=0)
rf_fixed.fit(X_train, y_train)


Evaluate it

In [ ]:
pred = rf_fixed.predict(X_test)

In [ ]:
accuracy_coordinates = metrics.accuracy_score(pred, y_test)
accuracy_coordinates

In [ ]:
kappa_coordinates = metrics.cohen_kappa_score(pred, y_test)
kappa_coordinates

In [ ]:
accuracy, kappa

In [ ]:
training_sample.columns.values

# Regressions 

In [ ]:
independent = training_sample_price[independent_variables]
X_train, X_test, y_train, y_test = model_selection.train_test_split(independent, training_sample_price["CENA"], test_size=.25, random_state=0)

In [ ]:
price_model = ensemble.RandomForestRegressor(random_state=0)
price_model.fit(X_train, y_train)

Evaluate it

In [ ]:
pred = price_model.predict(X_test)

In [ ]:
r_squared = metrics.r2_score(pred, y_test)
r_squared

In [ ]:
mae = metrics.mean_absolute_error(pred, y_test)
mae

In [ ]:
predicted  = model_selection.cross_val_predict(price_model, independent, training_sample_price["CENA"], cv=4)
residuals =  training_sample_price["CENA"] - predicted
maximum = np.max(np.abs(residuals))
training_sample_price.plot(residuals, figsize=(16, 16), markersize=.5, cmap="bwr", legend=True, vmin=-maximum, vmax=maximum)